In [1]:
# **Step 2: The Python Script (`train_snake.py`)**

import gymnasium as gym
from gymnasium import spaces
import numpy as np
import elastica as ea
from collections import defaultdict
from typing import Optional
from numpy.typing import NDArray
# from PyElastica.elastica.timestepper.symplectic_steppers import do_step
# from PyElastica.elastica.timestepper import extend_stepper_interface
from elastica.typing import RodType


from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.callbacks import BaseCallback

import matplotlib.pyplot as plt

import os

In [2]:

class BaseContinuumSnakeEnv(gym.Env):
    """
    Shared implementation for the PyElastica Continuum Snake Gymnasium environment.
    Subclasses configure how wavelength is handled in the action space.
    """
    metadata = {"render_modes": []}

    def __init__(self, obs_keys: Optional[list] = None):
        super().__init__()

        self._n_elem = 50
        self._torque_min = 1e-3
        self._torque_max = 8e-3
        self._torque_span = self._torque_max - self._torque_min

        action_space_low, action_space_high = self._action_space_bounds()
        self.action_space = spaces.Box(low=action_space_low, high=action_space_high, dtype=np.float32)

        if obs_keys is None:
            obs_keys = ["position", "velocity", "director"]
        self.obs_keys = obs_keys

        valid_keys = [
            "time",
            "avg_position",
            "avg_velocity",
            "curvature",
            "tangents",
            "position",
            "velocity",
            "director",
        ]
        for key in self.obs_keys:
            if key not in valid_keys:
                raise ValueError(f"Invalid observation key: {key}. Valid keys are: {valid_keys}")

        obs_size = self._calculate_obs_size()
        self.observation_space = spaces.Box(-np.inf, np.inf, shape=(obs_size,), dtype=np.float32)

        self.period = 2.0
        self.ratio_time = 11.0
        self.time_step = 1e-4
        self.rut_ratio = 1
        self.slip_velocity_tol = 1e-8

        self.start = np.zeros((3,))
        self.direction = np.array([0.0, 0.0, 1.0])
        self.normal = np.array([0.0, 1.0, 0.0])
        self.base_length = 0.35
        self.base_radius = self.base_length * 0.011
        self.density = 1000
        self.E = 1e6
        self.poisson_ratio = 0.5
        self.shear_modulus = self.E / (1.0 + self.poisson_ratio)

        self.timestepper = ea.PositionVerlet()

        self.current_time = 0.0
        self.state_dict = {
            "time": [],
            "avg_position": [],
            "avg_velocity": [],
            "curvature": [],
            "tangents": [],
            "position": [],
            "velocity": [],
            "director": [],
            "torque_coeffs": [],
            "wave_number": [],
            "forward": [],
            "lateral": [],
            "reward": [],
        }

        self._prev_com = None
        self._dir = np.array([0.0, 0.0, 1.0], dtype=np.float64)
        self.forward = 0.0
        self.lateral = 0.0

        target = np.array([1.0, 0.0, 1.0], dtype=np.float64)
        self.target_direction = target / (np.linalg.norm(target) + 1e-12)
        self.required_alignment_steps = 15 #3
        self.alignment_angle_tolerance = np.deg2rad(5.0)
        self._alignment_dot_threshold = np.cos(self.alignment_angle_tolerance)
        self.alignment_speed_tol = 1e-4
        self._alignment_streak = 0
        self.velocity_projection = 0.0
        self._last_velocity_vec = np.zeros(3, dtype=np.float64)

        self.reward_weights = {
            "forward_progress": 1.0,
            "lateral_penalty": 0.2,
            "curvature_penalty": 0.05,
            "energy_penalty": 2.0e4,
            "smoothness_penalty": 5.0e3,
            "alignment_bonus": 0.5,
            "streak_bonus": 1.0,
        }
        self._prev_torque_coeffs = None

        self.callback_data = defaultdict(list)

    def _action_space_bounds(self) -> tuple[NDArray[np.float32], NDArray[np.float32]]:
        raise NotImplementedError

    def _default_action(self) -> NDArray[np.float64]:
        raise NotImplementedError

    def _map_action_to_torque(self, action: NDArray[np.float64]) -> NDArray[np.float64]:
        action_arr = np.clip(np.asarray(action, dtype=np.float64), -1.0, 1.0)
        magnitudes = self._torque_max - self._torque_span * np.abs(action_arr)
        torques = np.sign(action_arr) * magnitudes
        torques[np.isclose(action_arr, 0.0)] = 0.0
        return torques.astype(np.float64)

    def _map_torque_to_action(self, torque: NDArray[np.float64]) -> NDArray[np.float64]:
        torque_arr = np.asarray(torque, dtype=np.float64)
        magnitudes = np.abs(torque_arr)
        clipped = np.clip(magnitudes, 0.0, self._torque_max)
        abs_action = (self._torque_max - clipped) / self._torque_span
        abs_action = np.clip(abs_action, 0.0, 1.0)
        return (abs_action * np.sign(torque_arr)).astype(np.float64)

    def _augment_action(self, action: NDArray[np.float64]) -> NDArray[np.float64]:
        raise NotImplementedError

    @staticmethod
    def _default_torque_coeffs() -> NDArray[np.float64]:
        return np.array([3.4e-3, 3.3e-3, 4.2e-3, 2.6e-3, 3.6e-3, 3.5e-3], dtype=np.float64)

    def _calculate_obs_size(self) -> int:
        n_elem = self._n_elem
        n_nodes = n_elem + 1
        key_sizes = {
            "time": 1,
            "avg_position": 3,
            "avg_velocity": 3,
            "curvature": (n_elem - 1) * 3,
            "tangents": n_elem * 3,
            "position": n_nodes * 3,
            "velocity": n_nodes * 3,
            "director": n_elem * 9,
        }
        return sum(key_sizes[key] for key in self.obs_keys)

    def _transition_model(self, b_coeff: NDArray[np.float64]):
        class SnakeSimulator(
            ea.BaseSystemCollection,
            ea.Constraints,
            ea.Forcing,
            ea.Damping,
            ea.CallBacks,
            ea.Contact,
        ):
            pass

        snake_sim = SnakeSimulator()
        snake_sim.append(self.shearable_rod)

        gravitational_acc = -9.80665
        snake_sim.add_forcing_to(self.shearable_rod).using(
            ea.GravityForces, acc_gravity=np.array([0.0, gravitational_acc, 0.0])
        )

        wave_length = b_coeff[-1]
        b_coeff_only = b_coeff[:-1]
        snake_sim.add_forcing_to(self.shearable_rod).using(
            ea.MuscleTorques,
            base_length=self.base_length,
            b_coeff=b_coeff_only,
            period=self.period,
            wave_number=2.0 * np.pi / wave_length,
            phase_shift=0.0,
            rest_lengths=self.shearable_rod.rest_lengths,
            ramp_up_time=self.rut_ratio * self.period,
            direction=self.normal,
            with_spline=True,
        )

        ground_plane = ea.Plane(
            plane_origin=np.array([0.0, -self.base_radius, 0.0]),
            plane_normal=self.normal,
        )
        snake_sim.append(ground_plane)
        froude = 0.1
        mu = self.base_length / (self.period * self.period * np.abs(gravitational_acc) * froude)
        kinetic_mu_array = np.array([mu, 1.5 * mu, 2.0 * mu])  # [forward, backward, sideways]
        static_mu_array = np.zeros(kinetic_mu_array.shape)
        snake_sim.detect_contact_between(self.shearable_rod, ground_plane).using(
            ea.RodPlaneContactWithAnisotropicFriction,
            k=1.0,
            nu=1e-6,
            slip_velocity_tol=self.slip_velocity_tol,
            static_mu_array=static_mu_array,
            kinetic_mu_array=kinetic_mu_array,
        )

        damping_constant = 2e-3
        snake_sim.dampen(self.shearable_rod).using(
            ea.AnalyticalLinearDamper,
            damping_constant=damping_constant,
            time_step=self.time_step,
        )

        if not hasattr(self, "callback_data") or self.callback_data is None:
            self.callback_data = defaultdict(list)
        rendering_fps = 60
        step_skip = int(1.0 / (rendering_fps * self.time_step))
        slip_velocity_tol = self.slip_velocity_tol

        class ContinuumSnakeCallBack(ea.CallBackBaseClass):
            def __init__(self, step_skip: int, callback_params: dict) -> None:
                ea.CallBackBaseClass.__init__(self)
                self.every = step_skip
                self.callback_params = callback_params

            def make_callback(
                self, system: ea.CosseratRod, time: float, current_step: int
            ) -> None:
                if current_step == 0:
                    return

                if current_step % self.every == 0:
                    self.callback_params["time"].append(time)
                    self.callback_params["step"].append(current_step)
                    self.callback_params["position"].append(
                        system.position_collection.copy()
                    )
                    self.callback_params["velocity"].append(
                        system.velocity_collection.copy()
                    )
                    self.callback_params["avg_velocity"].append(
                        system.compute_velocity_center_of_mass()
                    )
                    self.callback_params["center_of_mass"].append(
                        system.compute_position_center_of_mass()
                    )
                    self.callback_params["curvature"].append(system.kappa.copy())
                    self.callback_params["tangents"].append(system.tangents.copy())
                    self.callback_params["friction"].append(
                        self.get_slip_velocity(system).copy()
                    )
                    return

            def get_slip_velocity(self, system: RodType) -> NDArray[np.float64]:
                from elastica.contact_utils import (
                    _find_slipping_elements,
                    _node_to_element_velocity,
                )
                from elastica._linalg import _batch_dot, _batch_product_k_ik_to_ik

                axial_direction = system.tangents
                element_velocity = _node_to_element_velocity(
                    mass=system.mass, node_velocity_collection=system.velocity_collection
                )
                velocity_mag_along_axial_direction = _batch_dot(
                    element_velocity, axial_direction
                )
                velocity_along_axial_direction = _batch_product_k_ik_to_ik(
                    velocity_mag_along_axial_direction, axial_direction
                )
                return _find_slipping_elements(
                    velocity_along_axial_direction, slip_velocity_tol
                )

        snake_sim.collect_diagnostics(self.shearable_rod).using(
            ContinuumSnakeCallBack,
            step_skip=step_skip,
            callback_params=self.callback_data,
        )

        snake_sim.finalize()
        return snake_sim

    def _get_obs_value_from_rod(self, key: str):
        if key == "position":
            return self.shearable_rod.position_collection.ravel()
        if key == "velocity":
            return self.shearable_rod.velocity_collection.ravel()
        if key == "director":
            return self.shearable_rod.director_collection.ravel()
        if key == "curvature":
            return self.shearable_rod.kappa.ravel()
        if key == "tangents":
            return self.shearable_rod.tangents.ravel()
        if key == "avg_position":
            return self.shearable_rod.compute_position_center_of_mass()
        if key == "avg_velocity":
            return self.shearable_rod.compute_velocity_center_of_mass()
        if key == "time":
            return np.array([self.current_time])
        raise ValueError(f"Unknown observation key: {key}")

    def _get_obs(self):
        obs_parts = []
        use_state_dict = (
            len(self.obs_keys) > 0
            and self.obs_keys[0] in self.state_dict
            and len(self.state_dict[self.obs_keys[0]]) > 0
        )

        for key in self.obs_keys:
            if use_state_dict and key in self.state_dict and len(self.state_dict[key]) > 0:
                latest_value = self.state_dict[key][-1]
                if isinstance(latest_value, np.ndarray):
                    obs_parts.append(latest_value.ravel())
                else:
                    obs_parts.append(np.array([latest_value]))
            else:
                obs_parts.append(self._get_obs_value_from_rod(key))

        return np.concatenate(obs_parts).astype(np.float32)

    def reset(self, seed: Optional[int] = None, options: Optional[dict] = None):
        super().reset(seed=seed)

        self.current_time = 0.0
        self.state_dict = {
            "time": [],
            "avg_position": [],
            "avg_velocity": [],
            "curvature": [],
            "tangents": [],
            "position": [],
            "velocity": [],
            "director": [],
            "torque_coeffs": [],
            "wave_number": [],
            "forward": [],
            "lateral": [],
            "reward": [],
        }

        self._prev_com = None
        self._dir = np.array([0.0, 0.0, 1.0], dtype=np.float64)
        self.forward = 0.0
        self.lateral = 0.0
        self.reward = 0.0
        self.velocity_projection = 0.0
        self._alignment_streak = 0
        self._last_velocity_vec = np.zeros(3, dtype=np.float64)
        self._prev_torque_coeffs = None

        self.callback_data = defaultdict(list)

        self.shearable_rod = ea.CosseratRod.straight_rod(
            self._n_elem,
            self.start,
            self.direction,
            self.normal,
            self.base_length,
            self.base_radius,
            self.density,
            youngs_modulus=self.E,
            shear_modulus=self.shear_modulus,
        )

        reset_action = self._default_action()
        if options is not None and "action" in options:
            reset_action = np.asarray(options["action"], dtype=np.float64)
        reset_action = np.asarray(reset_action, dtype=np.float64)
        b_coeff = self._augment_action(reset_action)

        self.sim = self._transition_model(b_coeff)

        observation = self._get_obs()
        info = {}
        return observation, info

    def _forward_lateral_from_last_step(
        self, sim_time: float
    ) -> tuple[float, float, NDArray[np.float64]]:
        com = self.shearable_rod.compute_position_center_of_mass()
        if self._prev_com is None:
            self._prev_com = com.copy()
            return 0.0, 0.0, np.zeros(3, dtype=np.float64)

        disp = com - self._prev_com
        velocity_vec = disp / (sim_time + 1e-12)
        new_dir = velocity_vec / (np.linalg.norm(velocity_vec) + 1e-12)
        alpha = 1.0
        self._dir = (1 - alpha) * self._dir + alpha * new_dir
        self._dir /= np.linalg.norm(self._dir) + 1e-12

        forward = float(np.dot(velocity_vec, self._dir))
        lateral = float(
            np.linalg.norm(velocity_vec - np.dot(velocity_vec, self._dir) * self._dir)
        )

        self._prev_com = com.copy()
        return forward, lateral, velocity_vec

    def step(self, action):
        action = np.asarray(action, dtype=np.float64)
        b_coeff = self._augment_action(action)

        sim_time = (self.ratio_time + 0.001) * self.period
        total_steps = int(sim_time / self.time_step)

        self.sim = self._transition_model(b_coeff)

        prev_com_before = (
            self.shearable_rod.compute_position_center_of_mass().copy()
            if self._prev_com is None
            else self._prev_com.copy()
        )

        self.current_time = ea.integrate(
            self.timestepper,
            self.sim,
            sim_time,
            total_steps,
            self.current_time,
            progress_bar=False,
        )

        self.state_dict["time"].append(self.current_time)
        self.state_dict["avg_position"].append(
            self.shearable_rod.compute_position_center_of_mass()
        )
        self.state_dict["avg_velocity"].append(
            self.shearable_rod.compute_velocity_center_of_mass()
        )
        self.state_dict["curvature"].append(self.shearable_rod.kappa.copy())
        self.state_dict["tangents"].append(self.shearable_rod.tangents.copy())
        self.state_dict["position"].append(
            self.shearable_rod.position_collection.copy()
        )
        self.state_dict["velocity"].append(
            self.shearable_rod.velocity_collection.copy()
        )
        self.state_dict["director"].append(
            self.shearable_rod.director_collection.copy()
        )

        self._prev_com = prev_com_before
        self.forward, self.lateral, velocity_vec = self._forward_lateral_from_last_step(sim_time)
        self._last_velocity_vec = velocity_vec
        current_com = self.shearable_rod.compute_position_center_of_mass()
        self.velocity_projection = float(np.dot(velocity_vec, self.target_direction))

        speed = float(np.linalg.norm(velocity_vec))
        alignment = 0.0
        if speed > self.alignment_speed_tol:
            alignment = float(
                np.dot(velocity_vec / (speed + 1e-12), self.target_direction)
            )
        if speed > self.alignment_speed_tol and alignment >= self._alignment_dot_threshold:
            self._alignment_streak += 1
        else:
            self._alignment_streak = 0

        torque_coeffs = b_coeff[:-1].astype(np.float64).copy()
        forward_progress = float(np.dot(current_com - prev_com_before, self.target_direction))
        forward_term = self.reward_weights["forward_progress"] * forward_progress
        lateral_penalty = -self.reward_weights["lateral_penalty"] * float(self.lateral)
        curvature_array = self.shearable_rod.kappa
        curvature_magnitude = (
            float(np.mean(np.linalg.norm(curvature_array, axis=0))) if curvature_array.size > 0 else 0.0
        )
        curvature_penalty = -self.reward_weights["curvature_penalty"] * curvature_magnitude
        energy_penalty = -self.reward_weights["energy_penalty"] * float(np.linalg.norm(torque_coeffs) ** 2)

        smoothness_penalty = 0.0
        if self._prev_torque_coeffs is not None:
            delta_torque = torque_coeffs - self._prev_torque_coeffs
            smoothness_penalty = -self.reward_weights["smoothness_penalty"] * float(
                np.linalg.norm(delta_torque) ** 2
            )

        alignment_bonus = 0.0
        if speed > self.alignment_speed_tol and alignment > 0.0:
            alignment_bonus += self.reward_weights["alignment_bonus"] * alignment
            if self._alignment_streak >= self.required_alignment_steps:
                alignment_bonus += self.reward_weights["streak_bonus"]

        reward_terms = {
            "forward_progress": float(forward_term),
            "lateral_penalty": float(lateral_penalty),
            "curvature_penalty": float(curvature_penalty),
            "energy_penalty": float(energy_penalty),
            "smoothness_penalty": float(smoothness_penalty),
            "alignment_bonus": float(alignment_bonus),
        }

        self.reward = float(sum(reward_terms.values()))
        reward = float(self.reward)
        self._prev_torque_coeffs = torque_coeffs.copy()

        self.state_dict["torque_coeffs"].append(torque_coeffs.copy())
        self.state_dict["wave_number"].append(2.0 * np.pi / b_coeff[-1])
        self.state_dict["forward"].append(float(self.forward))
        self.state_dict["lateral"].append(float(self.lateral))
        self.state_dict["reward"].append(float(self.reward))

        max_sim_time = 300
        terminated_due_to_time = self.current_time >= max_sim_time
        terminated_due_to_alignment = self._alignment_streak >= self.required_alignment_steps
        # terminated_due_to_alignment = False
        terminated = terminated_due_to_time or terminated_due_to_alignment
        truncated = False

        observation = self._get_obs()

        info = {}
        info["reward"] = float(reward)
        info["reward_terms"] = reward_terms
        info["forward_speed"] = float(self.forward)
        info["lateral_speed"] = float(self.lateral)
        info["velocity_projection"] = float(self.velocity_projection)
        info["forward_progress"] = forward_progress
        info["speed"] = speed
        info["alignment"] = alignment
        info["alignment_streak"] = int(self._alignment_streak)
        info["alignment_goal_met"] = bool(terminated_due_to_alignment)
        info["position"] = current_com.astype(float)
        info["heading_dir"] = self._dir.astype(float)
        info["current_time"] = float(self.current_time)

        return observation, reward, terminated, truncated, info

    def close(self):
        pass


class FixedWavelengthContinuumSnakeEnv(BaseContinuumSnakeEnv):
    def __init__(self, fixed_wavelength: float, obs_keys: Optional[list] = None):
        self.fixed_wavelength = float(fixed_wavelength)
        super().__init__(obs_keys=obs_keys)

    def _action_space_bounds(self) -> tuple[NDArray[np.float32], NDArray[np.float32]]:
        low = -np.ones(6, dtype=np.float32)
        high = np.ones(6, dtype=np.float32)
        return low, high

    def _default_action(self) -> NDArray[np.float64]:
        torques = self._default_torque_coeffs()
        return self._map_torque_to_action(torques)

    def _augment_action(self, action: NDArray[np.float64]) -> NDArray[np.float64]:
        if action.shape[-1] != 6:
            raise ValueError("Expected 6 torque coefficients for fixed-wavelength control.")
        torques = self._map_action_to_torque(action)
        return np.append(torques, self.fixed_wavelength)


class VariableWavelengthContinuumSnakeEnv(BaseContinuumSnakeEnv):
    def __init__(
        self,
        obs_keys: Optional[list] = None,
        wavelength_bounds: tuple[float, float] = (-1.5, 1.5),
        default_wavelength: float = 1.0,
    ):
        self.wavelength_bounds = wavelength_bounds
        self.default_wavelength = default_wavelength
        super().__init__(obs_keys=obs_keys)

    def _action_space_bounds(self) -> tuple[NDArray[np.float32], NDArray[np.float32]]:
        low = -np.ones(7, dtype=np.float32)
        high = np.ones(7, dtype=np.float32)
        low[-1] = np.float32(self.wavelength_bounds[0])
        high[-1] = np.float32(self.wavelength_bounds[1])
        return low, high

    def _default_action(self) -> NDArray[np.float64]:
        torques = self._default_torque_coeffs()
        torque_actions = self._map_torque_to_action(torques)
        return np.append(torque_actions, self.default_wavelength)

    def _augment_action(self, action: NDArray[np.float64]) -> NDArray[np.float64]:
        if action.shape[-1] != 7:
            raise ValueError(
                "Expected 7 action dimensions (6 torque coefficients + wavelength)."
            )
        action = np.asarray(action, dtype=np.float64).copy()
        torques = self._map_action_to_torque(action[:-1])
        min_w, max_w = self.wavelength_bounds
        wavelength = float(np.clip(action[-1], min_w, max_w))
        return np.append(torques, wavelength)


In [3]:
# Create environment with fixed wavelength (action space size 6)
# To allow modifiable wavelength (action space size 7), use VariableWavelengthContinuumSnakeEnv.

# Customize the observation/state space by editing this list of keys.
# Set to None to fall back to the default keys defined in BaseContinuumSnakeEnv.
custom_obs_keys = [
    "position",
    "velocity",
    "curvature",
]

env = FixedWavelengthContinuumSnakeEnv(
    fixed_wavelength=1.0,
    obs_keys=custom_obs_keys,
    
)
env.reset()

# Verify action space is now size 6
print(f"Action space shape: {env.action_space.shape}")
print(f"Fixed wavelength: {env.fixed_wavelength}")
print(f"Observation space shape: {env.observation_space.shape}")
print(f"Observation keys: {env.obs_keys}")

env.period = 1
env.ratio_time = 0.01
env.rut_ratio = 0.001


Action space shape: (6,)
Fixed wavelength: 1.0
Observation space shape: (453,)
Observation keys: ['position', 'velocity', 'curvature']


constant input

In [5]:

# Run 30 steps using the default action
num_steps = 30

# Get default action based on environment configuration
# default_action = np.array([3.4e-3, 3.3e-3, 4.2e-3, 2.6e-3, 3.6e-3, 3.5e-3], dtype=np.float32)
# default_action = np.array([3e-3, 3e-3, 3e-3, 3e-3, 3e-3, 3e-3], dtype=np.float32)
# default_action = np.array([1e-2, 1e-2, 1e-2, 1e-2, 1e-2, 1e-2], dtype=np.float32)
# default_action = np.array([5e-3, 5e-3, 5e-3, 5e-3, 5e-3, 5e-3], dtype=np.float32)
default_action = np.array([-5e-3, 5e-3, 5e-3, 5e-3, -5e-3, -5e-3], dtype=np.float32)
# default_action = np.array([-9.9e-3, 9.9e-3, 9.9e-3, 9.9e-3, -9.9e-3, -9.9e-3], dtype=np.float32)
# default_action = np.array([-1e-2, 1e-2, 1e-2, 1e-2, -1e-2, -1e-2], dtype=np.float32)
# default_action = np.array([5e-4, 5e-4, 5e-4, 5e-4, 5e-4, 5e-4], dtype=np.float32)
# default_action = np.array([1e-3, 1e-3, 1e-3, 1e-3, 1e-3, 1e-3], dtype=np.float32)
# default_action = np.array([3e-3, 5e-3, 4e-3, 3.5e-3, 3e-3, 2e-3], dtype=np.float32)
# default_action = np.array([5e-3, 4e-3, 3.5e-3, 3e-3, 2e-3, 1e-3], dtype=np.float32)



action = env._map_torque_to_action(default_action)
print(action)

[-0.42857144  0.42857144  0.42857144  0.42857144 -0.42857144 -0.42857144]


In [6]:

# Run simulation steps
for i in range(num_steps):
    obs, reward, terminated, truncated, info = env.step(action)
    print(f"Step {i+1}/{num_steps}: time={info['current_time']:.2f}s, "
          f"forward={info['forward_speed']:.4f} m/s, "
          f"lateral={info['lateral_speed']:.4f} m/s, "
          f"reward={info['reward']:.4f}")
    
    if terminated or truncated:
        print(f"Simulation ended at step {i+1}")
        break

Step 1/30: time=0.01s, forward=0.0020 m/s, lateral=0.0000 m/s, reward=-3.1667
Step 2/30: time=0.02s, forward=0.0047 m/s, lateral=0.0000 m/s, reward=-3.1811
Step 3/30: time=0.03s, forward=0.0074 m/s, lateral=0.0000 m/s, reward=-3.4748
Step 4/30: time=0.04s, forward=0.0038 m/s, lateral=0.0000 m/s, reward=-3.5757
Step 5/30: time=0.05s, forward=0.0076 m/s, lateral=0.0000 m/s, reward=-3.6263
Step 6/30: time=0.07s, forward=0.0143 m/s, lateral=0.0000 m/s, reward=-3.6433
Step 7/30: time=0.08s, forward=0.0206 m/s, lateral=0.0000 m/s, reward=-3.6559
Step 8/30: time=0.09s, forward=0.0269 m/s, lateral=0.0000 m/s, reward=-3.6730
Step 9/30: time=0.10s, forward=0.0324 m/s, lateral=0.0000 m/s, reward=-3.7109
Step 10/30: time=0.11s, forward=0.0375 m/s, lateral=0.0000 m/s, reward=-3.7454
Step 11/30: time=0.12s, forward=0.0413 m/s, lateral=0.0000 m/s, reward=-3.7563
Step 12/30: time=0.13s, forward=0.0431 m/s, lateral=0.0000 m/s, reward=-3.7346
Step 13/30: time=0.14s, forward=0.0441 m/s, lateral=0.0000 m/

visulization

In [ ]:
from Utilities.visualization import visualization

v1 = visualization()


In [9]:

# Visualize trajectory in 2D (X vs Z)
avg_positions = np.asarray(env.state_dict['avg_position'])
# v1.trajectory_2d(avg_positions[:, 0], avg_positions[:, 2])

# Visualize velocity norm history
avg_velocities = np.asarray(env.state_dict['avg_velocity'])
# v1.velocity_norm_history(env.state_dict['time'], avg_velocities)

# Visualize trajectory in 3D
# v1.trajectory_3d(avg_positions)

# v1.trajectory_2d(avg_positions[:, 0], avg_positions[:, 2])


# plt.plot(env.state_dict['time'], env.state_dict['torque_coeffs'])
# plt.plot(env.state_dict['time'], env.state_dict['reward'])


# Visualize current snake shape
# v1.current_shape_3d(env.shearable_rod)

# Plot video from callback data
# v1.plot_video(env.callback_data)
# v1.plot_video_limited( 
#                       env.callback_data,
#                       "video_limited1.mp4",
#                       1500
#                       )
v1.plot_video_auto_limits(
    env.callback_data,
    "video_auto_limits.mp4",
    1500,
)



Plotting limited video: using 1500 of 3283 frames, 27.28 fps, 54.98 seconds duration


100%|██████████| 1499/1499 [00:16<00:00, 88.43it/s]


check on a dictionary

In [10]:
# Print shapes of all keys in callback_data
print("Shapes of all keys in env.callback_data:")
print("=" * 50)
for key in sorted(env.callback_data.keys()):
    data = env.callback_data[key]
    if len(data) > 0:
        shape = np.asarray(data).shape
        print(f"{key:20s}: {shape}")
    else:
        print(f"{key:20s}: (empty)")
print("=" * 50)

Shapes of all keys in env.callback_data:
avg_velocity        : (19, 3)
center_of_mass      : (19, 3)
curvature           : (19, 3, 49)
friction            : (19, 50)
position            : (19, 3, 51)
step                : (19,)
tangents            : (19, 3, 50)
time                : (19,)
velocity            : (19, 3, 51)


In [9]:
log_path = os.path.join("Training", "Logs")
model = PPO(
    "MlpPolicy",
    env,
    verbose=1,
    # tensorboard_log=log_path
)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [14]:
# Custom callback to print rewards during training
class RewardCallback(BaseCallback):
    """
    Callback for printing reward information during training.
    Prints episode rewards and, optionally, step-level metrics.
    """
    def __init__(
        self,
        verbose: int = 0,
        print_freq: int = 5,
        step_print_interval: Optional[int] = None,
        step_info_keys: Optional[list[str]] = None,
    ):
        super(RewardCallback, self).__init__(verbose)
        self.print_freq = print_freq
        self.step_print_interval = step_print_interval
        self.step_info_keys = step_info_keys or []
        self.episode_rewards = []
        self.episode_lengths = []
        self.step_rewards = []
        self.episode_count = 0

    def _on_step(self) -> bool:
        rewards = self.locals.get('rewards', [0])
        infos = self.locals.get('infos', [{}])

        if len(rewards) > 0:
            step_reward = float(np.mean(rewards))
            self.step_rewards.append(step_reward)

            if self.step_print_interval and self.num_timesteps % self.step_print_interval == 0:
                info_dict = infos[0] if len(infos) > 0 and isinstance(infos[0], dict) else {}
                info_bits = []
                for key in self.step_info_keys:
                    if key in info_dict:
                        info_bits.append(f"{key}={info_dict[key]}")
                info_suffix = f" | {'; '.join(info_bits)}" if info_bits else ""
                print(f"[Step {self.num_timesteps}] reward={step_reward:.6f}{info_suffix}")

        if len(infos) > 0 and isinstance(infos[0], dict):
            if 'episode' in infos[0]:
                episode_info = infos[0]['episode']
                if episode_info is not None:
                    episode_reward = episode_info.get('r', 0)
                    episode_length = episode_info.get('l', 0)
                    self.episode_rewards.append(episode_reward)
                    self.episode_lengths.append(episode_length)
                    self.episode_count += 1

                    print(
                        f"\n[Episode {self.episode_count}] "
                        f"Reward = {episode_reward:.4f}, "
                        f"Length = {episode_length} steps"
                    )

                    if self.episode_count % self.print_freq == 0:
                        avg_reward = np.mean(self.episode_rewards[-self.print_freq:])
                        print(f"  -> Average reward (last {self.print_freq} episodes) = {avg_reward:.4f}\n")

        return True


In [21]:
total_training_timesteps = 2000
# total_training_timesteps = 50_000
print(f"Starting training for {total_training_timesteps} timesteps...")

# Create callback to print rewards
reward_callback = RewardCallback(print_freq=5, step_print_interval=20)  # Print every 5 steps

# Train with callback
model.learn(total_timesteps=total_training_timesteps, callback=reward_callback)

Starting training for 2000 timesteps...
Final time of simulation is :  0.010999999999999987
Final time of simulation is :  0.022000000000000193
Final time of simulation is :  0.03300000000000044
Final time of simulation is :  0.044000000000000046
Final time of simulation is :  0.05499999999999965
Final time of simulation is :  0.06599999999999974
Final time of simulation is :  0.07700000000000086
Final time of simulation is :  0.08800000000000198
Final time of simulation is :  0.0990000000000031
Final time of simulation is :  0.11000000000000422
Final time of simulation is :  0.12100000000000534
Final time of simulation is :  0.13200000000000645
Final time of simulation is :  0.14300000000000757
Final time of simulation is :  0.15400000000000869
Final time of simulation is :  0.1650000000000098
Final time of simulation is :  0.17600000000001093
Final time of simulation is :  0.18700000000001205
Final time of simulation is :  0.19800000000001317
Final time of simulation is :  0.20900000

In [5]:
import os


In [ ]:

save_dir = os.path.join("Training", "Saved_Models")
os.makedirs(save_dir, exist_ok=True)        # create folders the first time

model_path = os.path.join(save_dir, "PPO_Snake_Model")
model.save(model_path)
print(f"Training finished! Model saved to {model_path}")

load and run trained models

In [6]:

# Recreate the environment (since it was closed in Cell 14)


# Load the trained model
model_path = os.path.join("Training", "Saved_Models", "PPO_Snake_Model")
model = PPO.load(model_path, env=env)
print(f"Model loaded from {model_path}")


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Model loaded from Training/Saved_Models/PPO_Snake_Model


In [ ]:

# Create environment with fixed wavelength (action space size 6)
# To allow modifiable wavelength (action space size 7), use VariableWavelengthContinuumSnakeEnv.

# Customize the observation/state space by editing this list of keys.
# Set to None to fall back to the default keys defined in BaseContinuumSnakeEnv.
custom_obs_keys = [
    "position",
    "velocity",
    "curvature",
]

env = FixedWavelengthContinuumSnakeEnv(
    fixed_wavelength=1.0,
    obs_keys=custom_obs_keys,
    
)
# Verify action space is now size 6
print(f"Action space shape: {env.action_space.shape}")
print(f"Fixed wavelength: {env.fixed_wavelength}")
print(f"Observation space shape: {env.observation_space.shape}")
print(f"Observation keys: {env.obs_keys}")

env.period = 1
env.ratio_time = 0.01
env.rut_ratio = 0.001



In [7]:

# Reset the environment
obs, _ = env.reset()

In [8]:

# Run a few steps using the trained model
num_steps = 5000  # Change this to run more or fewer steps
print(f"\nRunning {num_steps} steps with trained model...")
print("=" * 70)

import io, contextlib

total_reward = 0.0
for i in range(num_steps):
    # Get action from the trained model (deterministic=True for consistent results)
    action, _states = model.predict(obs, deterministic=True)
    # action = np.array([3.4e-3, 3.3e-3, 4.2e-3, 2.6e-3, 3.6e-3, 3.5e-3], dtype=np.float32)
    
    
    # Step the environment
    with contextlib.redirect_stdout(io.StringIO()):
        obs, reward, terminated, truncated, info = env.step(action)
    
    # Accumulate reward
    total_reward += reward
    
    # Print step information (including action)
    action_str = np.array2string(action, precision=6, suppress_small=True)
    status_line = (f"Step {i+1}/{num_steps} | "
                   f"Time: {info['current_time']:.2f}s | "
                   f"Forward: {info['forward_speed']:.4f} m/s | "
                   f"Lateral: {info['lateral_speed']:.4f} m/s | "
                   f"Reward: {reward:.4f} | "
                   f"Pos: [{info['position'][0]:.3f}, {info['position'][1]:.3f}, {info['position'][2]:.3f}] | "
                   f"Action: {action_str}")
    print(status_line)

    # Check if episode ended
    if terminated or truncated:
        print(f"Episode ended at step {i+1}")
        obs, _ = env.reset()
        print("Environment reset for next episode")

# Print summary after loop completes
print(f"\nTotal reward over {num_steps} steps: {total_reward:.4f}")
print(f"Average reward per step: {total_reward/num_steps:.4f}")



Running 5000 steps with trained model...
Step 1/5000 | Time: 0.01s | Forward: 0.0006 m/s | Lateral: 0.0000 m/s | Reward: -0.1973 | Pos: [-0.000, 0.000, 0.175] | Action: [-0.866362  1.       -1.        1.        1.       -1.      ]
Step 2/5000 | Time: 0.02s | Forward: 0.0015 m/s | Lateral: 0.0000 m/s | Reward: -0.2291 | Pos: [-0.000, 0.000, 0.175] | Action: [-0.936161  0.896351  1.        1.        1.       -1.      ]
Step 3/5000 | Time: 0.03s | Forward: 0.0042 m/s | Lateral: 0.0000 m/s | Reward: -0.2855 | Pos: [-0.000, 0.000, 0.175] | Action: [ 1.        1.        1.        0.864591 -1.        1.      ]
Step 4/5000 | Time: 0.04s | Forward: 0.0079 m/s | Lateral: 0.0000 m/s | Reward: -0.1870 | Pos: [-0.000, 0.000, 0.175] | Action: [ 1.  1. -1.  1. -1.  1.]
Step 5/5000 | Time: 0.05s | Forward: 0.0045 m/s | Lateral: 0.0000 m/s | Reward: -1.2523 | Pos: [-0.000, 0.000, 0.175] | Action: [ 0.173351  1.       -1.        1.       -1.        1.      ]
Step 6/5000 | Time: 0.07s | Forward: 0.0033 

In [ ]:

# Run a few steps using the trained model
num_steps = 5  # Change this to run more or fewer steps
print(f"\nRunning {num_steps} steps with trained model...")
print("=" * 70)

import io, contextlib

total_reward = 0.0
for i in range(num_steps):
    # Get action from the trained model (deterministic=True for consistent results)
    action, _states = model.predict(obs, deterministic=True)
    # action = np.array([3.4e-3, 3.3e-3, 4.2e-3, 2.6e-3, 3.6e-3, 3.5e-3], dtype=np.float32)
    
    
    # Step the environment
    with contextlib.redirect_stdout(io.StringIO()):
        obs, reward, terminated, truncated, info = env.step(action)
    
    # Accumulate reward
    total_reward += reward
    
    # Print step information (including action)
    action_str = np.array2string(action, precision=6, suppress_small=True)
    status_line = (f"Step {i+1}/{num_steps} | "
                   f"Time: {info['current_time']:.2f}s | "
                   f"Forward: {info['forward_speed']:.4f} m/s | "
                   f"Lateral: {info['lateral_speed']:.4f} m/s | "
                   f"Reward: {reward:.4f} | "
                   f"Pos: [{info['position'][0]:.3f}, {info['position'][1]:.3f}, {info['position'][2]:.3f}] | "
                   f"Action: {action_str}")
    print(status_line)

    # Check if episode ended
    if terminated or truncated:
        print(f"Episode ended at step {i+1}")
        obs, _ = env.reset()
        print("Environment reset for next episode")

# Print summary after loop completes
print(f"\nTotal reward over {num_steps} steps: {total_reward:.4f}")
print(f"Average reward per step: {total_reward/num_steps:.4f}")


In [ ]:

# 4. Save the trained model
model_path = os.path.join("Training", "Saved_Models", "PPO_Snake_Model")
model.save(model_path)
print(f"Training finished! Model saved to {model_path}")

# --- How to load and run the trained model ---
del model
# model = PPO.load(model_path, env=env)
# obs, _ = env.reset()
# for i in range(200):
#     action, _ = model.predict(obs, deterministic=True)
#     obs, reward, terminated, truncated, info = env.step(action)
#     if terminated or truncated:
#         print(f"Episode finished at step {i}")
#         obs, _ = env.reset()

env.close()

In [ ]:
class visualization:
    def __init__(self):
        pass
        
    def trajectory_2d(self,
                      x,
                      z,
                      save_figure=False,
                      filename='trajectory_2d.png'
        ):

        plt.figure()
        plt.plot(x, z, marker='o', linestyle='-')
        plt.xlabel('X')
        plt.ylabel('Z')
        plt.title('X vs Z Endpoint Trajectory')
        plt.grid(True)
        plt.show()
        if save_figure:
            plt.savefig(filename)
        
    def trajectory_3d(self,
                      pts,
                      save_figure=False,
                      filename='trajectory_3d.png'
        ):
        pts = np.asarray(pts)
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.plot(pts[:, 0], pts[:, 1], pts[:, 2], marker='o', linewidth=1)
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.set_title('Trajectory')
        if save_figure:
            plt.savefig(filename)
            
    def current_shape_3d(self,
                        shearable_rod,
                        save_figure=False,
                        filename='current_shape.png'
        ):
        from matplotlib.colors import to_rgb

        fig = plt.figure(figsize=(10, 8), frameon=True, dpi=150)
        ax = fig.add_subplot(111, projection='3d')

        ax.plot(
            shearable_rod.position_collection[0, ...],
            shearable_rod.position_collection[2, ...],
            shearable_rod.position_collection[1, ...],
            c=to_rgb("xkcd:bluish"),
            label="n=" + str(shearable_rod.n_elems),
        )
        ax.set_xlabel('X')
        ax.set_ylabel('Z')
        ax.set_zlabel('Y')
        
        # Set equal aspect ratio for proper scaling
        # Get current axis limits
        xlims = ax.get_xlim()
        ylims = ax.get_ylim()
        zlims = ax.get_zlim()
        
        # Calculate the range for each axis
        x_range = xlims[1] - xlims[0]
        y_range = ylims[1] - ylims[0]
        z_range = zlims[1] - zlims[0]
        
        # Set equal aspect ratio based on the largest range
        max_range = max(x_range, y_range, z_range)
        ax.set_box_aspect([x_range/max_range, y_range/max_range, z_range/max_range])
        
        plt.show()
        if save_figure:
            plt.savefig(filename)
        
    def current_shape_2d(self,
                        shearable_rod,
                        save_figure=False,
                        filename='current_shape.png'
        ):
        from matplotlib.colors import to_rgb

        fig = plt.figure(figsize=(10, 8), frameon=True, dpi=150)
        ax = fig.add_subplot(111)

        ax.plot(
            shearable_rod.position_collection[2, ...],
            shearable_rod.position_collection[0, ...],
            c=to_rgb("xkcd:bluish"),
            label="n=" + str(shearable_rod.n_elems),
        )
        ax.set_xlabel('Z')
        ax.set_ylabel('X')
        plt.show()
        if save_figure:
            plt.savefig(filename)

        
    def velocity_norm_history(self,
                              time: np.ndarray,
                              velocity: np.ndarray,  # shape should be (N, 3)
                              save_figure=False,
                              filename='velocity_norm_history.png'
        ):
        velocity_norm = np.linalg.norm(velocity, axis=1)
        plt.figure()
        plt.plot(time, velocity_norm)
        plt.xlabel('Time')
        plt.ylabel('Velocity Norm')
        plt.title('Velocity Norm History')
        if save_figure:
            plt.savefig(filename)        
        
        
    def plot_video(
        self,
        plot_params: dict,
        video_name: str = "video.mp4",
        fps: int = 60,
        xlim: tuple[float, float] = (0, 4),
        ylim: tuple[float, float] = (-1, 1),
    ) -> None:  # (time step, x/y/z, node)
        import matplotlib.animation as manimation
        from tqdm import tqdm
        positions_over_time = np.array(plot_params["position"])

        num_frames = len(plot_params["time"])
        duration = num_frames / fps
        print(f"Plotting video: {num_frames} frames, {duration:.2f} seconds duration")
        
        FFMpegWriter = manimation.writers["ffmpeg"]
        metadata = dict(title="Movie Test", artist="Matplotlib", comment="Movie support!")
        writer = FFMpegWriter(fps=fps, metadata=metadata)
        fig = plt.figure(figsize=(10, 8), frameon=True, dpi=150)
        ax = fig.add_subplot(111)
        ax.set_xlim(*xlim)
        ax.set_ylim(*ylim)
        ax.set_xlabel("z [m]", fontsize=16)
        ax.set_ylabel("x [m]", fontsize=16)
        rod_lines_2d = ax.plot(positions_over_time[0][2], positions_over_time[0][0])[0]
        # plt.axis("equal")
        with writer.saving(fig, video_name, dpi=150):
            for time in tqdm(range(1, len(plot_params["time"]))):
                rod_lines_2d.set_xdata([positions_over_time[time][2]])
                rod_lines_2d.set_ydata([positions_over_time[time][0]])
                writer.grab_frame()

        # Be a good boy and close figures
        # https://stackoverflow.com/a/37451036
        # plt.close(fig) alone does not suffice
        # See https://github.com/matplotlib/matplotlib/issues/8560/
        plt.close(plt.gcf())

    def plot_video_limited(
        self,
        plot_params: dict,
        video_name: str = "video_limited.mp4",
        max_samples: int = 3000,
        xlim: tuple[float, float] = (0, 4),
        ylim: tuple[float, float] = (-1, 1),
    ) -> None:
        import matplotlib.animation as manimation
        from tqdm import tqdm

        times = np.asarray(plot_params["time"])
        positions_over_time = np.asarray(plot_params["position"])

        if times.ndim == 0 or len(times) == 0:
            raise ValueError("plot_params['time'] must contain at least one entry.")
        if positions_over_time.shape[0] != len(times):
            raise ValueError(
                "First dimension of plot_params['position'] must match length of plot_params['time']."
            )

        total_available = len(times)
        if total_available <= max_samples:
            indices = np.arange(total_available, dtype=int)
        else:
            indices = np.linspace(0, total_available - 1, max_samples)
            indices = np.round(indices).astype(int)
            indices[0] = 0
            indices[-1] = total_available - 1
            indices = np.unique(indices)
            if indices[0] != 0:
                indices = np.insert(indices, 0, 0)
            if indices[-1] != total_available - 1:
                indices = np.append(indices, total_available - 1)

        subsampled_times = times[indices]
        subsampled_positions = positions_over_time[indices]
        num_frames = len(subsampled_times)

        if num_frames == 0:
            raise ValueError("No frames selected for plotting.")

        total_time = float(subsampled_times[-1] - subsampled_times[0])
        if total_time > 0.0 and num_frames > 0:
            computed_fps = num_frames / total_time
        else:
            computed_fps = 1.0

        if computed_fps <= 0.0:
            computed_fps = 1.0

        duration = num_frames / computed_fps
        print(
            f"Plotting limited video: using {num_frames} of {total_available} frames, "
            f"{computed_fps:.2f} fps, {duration:.2f} seconds duration"
        )

        FFMpegWriter = manimation.writers["ffmpeg"]
        metadata = dict(title="Movie Test", artist="Matplotlib", comment="Movie support!")
        writer = FFMpegWriter(fps=computed_fps, metadata=metadata)
        fig = plt.figure(figsize=(10, 8), frameon=True, dpi=150)
        ax = fig.add_subplot(111)
        ax.set_xlim(*xlim)
        ax.set_ylim(*ylim)
        ax.set_xlabel("z [m]", fontsize=16)
        ax.set_ylabel("x [m]", fontsize=16)
        rod_lines_2d = ax.plot(
            subsampled_positions[0][2], subsampled_positions[0][0]
        )[0]
        with writer.saving(fig, video_name, dpi=150):
            for frame_idx in tqdm(range(1, num_frames)):
                rod_lines_2d.set_xdata([subsampled_positions[frame_idx][2]])
                rod_lines_2d.set_ydata([subsampled_positions[frame_idx][0]])
                writer.grab_frame()

        plt.close(plt.gcf())

    def plot_video_auto_limits(
        self,
        plot_params: dict,
        video_name: str = "video_auto_limits.mp4",
        max_samples: int = 3000,
        padding_ratio: float = 0.05,
    ) -> None:
        positions_over_time = np.asarray(plot_params["position"])

        if positions_over_time.shape[0] == 0:
            raise ValueError("plot_params['position'] must contain at least one entry.")

        if positions_over_time.ndim < 2:
            raise ValueError("plot_params['position'] must have at least two dimensions.")

        if positions_over_time.shape[1] == 3:
            oriented_positions = positions_over_time
        elif positions_over_time.shape[-1] == 3:
            oriented_positions = np.moveaxis(positions_over_time, -1, 1)
        else:
            raise ValueError(
                "plot_params['position'] must include a coordinate axis of length 3."
            )

        num_frames = oriented_positions.shape[0]
        reshaped_positions = oriented_positions.reshape(num_frames, 3, -1)
        z_values = reshaped_positions[:, 2, :].ravel()
        x_values = reshaped_positions[:, 0, :].ravel()

        finite_mask = np.isfinite(z_values) & np.isfinite(x_values)
        if not np.any(finite_mask):
            raise ValueError(
                "plot_params['position'] must contain finite values for x and z coordinates."
            )

        z_values = z_values[finite_mask]
        x_values = x_values[finite_mask]

        z_min = float(z_values.min())
        z_max = float(z_values.max())
        x_min = float(x_values.min())
        x_max = float(x_values.max())

        if np.isclose(z_min, z_max):
            z_center = 0.5 * (z_min + z_max)
            z_half_range = max(abs(z_center) * padding_ratio, 1e-3)
            xlim = (z_center - z_half_range, z_center + z_half_range)
        else:
            padding_z = padding_ratio * (z_max - z_min)
            xlim = (z_min - padding_z, z_max + padding_z)

        if np.isclose(x_min, x_max):
            x_center = 0.5 * (x_min + x_max)
            x_half_range = max(abs(x_center) * padding_ratio, 1e-3)
            ylim = (x_center - x_half_range, x_center + x_half_range)
        else:
            padding_x = padding_ratio * (x_max - x_min)
            ylim = (x_min - padding_x, x_max + padding_x)

        self.plot_video_limited(
            plot_params=plot_params,
            video_name=video_name,
            max_samples=max_samples,
            xlim=xlim,
            ylim=ylim,
        )